# Pipelines with AzureML Components

In [1]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient


credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<Subscription ID>"
    resource_group = "<ResourceGroup Name>"
    workspace = "<<WorkspaceName>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
print(ml_client)

We could not find config.json in: . or in its parent directories. 
MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f6bfd8aad90>,
         subscription_id=25758354-2fe7-426a-be0e-1ad20058340d,
         resource_group_name=azureml-live,
         workspace_name=ftaliveazureml)


In [2]:
#Get the compute target
cluster_name = 'cpu-cluster'
compute_target = ml_client.compute.get(name=cluster_name)

In [6]:
##Pipelines with YAML Components
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml import Input


parent_dir = "040_scripts"

# 1. Load components
prepare_data = load_component(path=parent_dir + "/prep_data.yml")
train_model = load_component(path=parent_dir + "/train_model.yml")
register_model = load_component(path=parent_dir+ "/register_model.yml")



# 2. Construct pipeline
@pipeline()
def diabetes_data_regression(pipeline_job_input):
    """Diabetes Training Sample Pipeline"""
    prepare_sample_data = prepare_data(raw_data=pipeline_job_input)
    train_model_data = train_model(
        training_data=prepare_sample_data.outputs.prep_data, learning_rate = 0.01
    )
    register_train_model = register_model(model_input_path = train_model_data.outputs.model_output, model_name = "Diabetest-LGBM-Model")


    return{
        "pipeline_job_prepped_data": prepare_sample_data.outputs.prep_data,
        "pipeline_job_trained_model": train_model_data.outputs.model_output
    }


pipeline_job = diabetes_data_regression( Input(type= "uri_file", path = "azureml:diabetes-dataset-uri-file:3", mode= "ro_mount"))
pipeline_job.outputs.pipeline_job_prepped_data.mode = "rw_mount"
# set pipeline level compute
pipeline_job.settings.default_compute = compute_target.name

In [7]:
# submit pipeline to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline-sample"
)


Uploading src (0.0 MBs): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4207/4207 [00:00<00:00, 64478.00it/s]


In [8]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

RunId: upbeat_crayon_r97cgf9v15
Web View: https://ml.azure.com/runs/upbeat_crayon_r97cgf9v15?wsid=/subscriptions/25758354-2fe7-426a-be0e-1ad20058340d/resourcegroups/azureml-live/workspaces/ftaliveazureml

Streaming logs/azureml/executionlogs.txt

[2022-06-22 08:59:35Z] Submitting 1 runs, first five are: 9cbfdc63:a76331b6-67c1-40b7-935f-6a01b8ee92e9
[2022-06-22 09:00:30Z] Completing processing run id a76331b6-67c1-40b7-935f-6a01b8ee92e9.
[2022-06-22 09:00:31Z] Submitting 1 runs, first five are: fcca4c52:839e8684-8c42-47e1-a8bb-db422e1bf034
[2022-06-22 09:01:22Z] Completing processing run id 839e8684-8c42-47e1-a8bb-db422e1bf034.
[2022-06-22 09:01:22Z] Submitting 1 runs, first five are: d45bf959:10509ded-705f-4596-ab20-37a1eee17e22
[2022-06-22 09:01:49Z] Completing processing run id 10509ded-705f-4596-ab20-37a1eee17e22.

Execution Summary
RunId: upbeat_crayon_r97cgf9v15
Web View: https://ml.azure.com/runs/upbeat_crayon_r97cgf9v15?wsid=/subscriptions/25758354-2fe7-426a-be0e-1ad20058340d/re

Refer more on Pipelines here: https://github.com/Azure/azureml-examples/tree/main/sdk/jobs/pipelines